In [1]:
from shapely.geometry import Point
import pandas as pd
import datetime
import time
import requests
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import folium.plugins as plugins
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

In [2]:
tornado_calls = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
tornado_incidents = pd.read_csv('../data/Data_Tornado.csv', skiprows = 3)
damage_points = gpd.read_file('../data/damage_points.geojson')
path_polygons = gpd.read_file('../data/path_polygons.geojson')
tornado_paths = gpd.read_file('../data/tornado_paths.geojson')

In [3]:
tornado_calls = tornado_calls.rename(columns = {'Seizure DateTime': 'time', 'CallTypeId': 'call', 'ALI Latitude': 'lat', 'ALI Longitude': 'lng', 'Cell Tower Address': 'tower_addy' })
tornado_incidents = tornado_incidents.rename(columns = {'IncidentDate': 'time', 'Location':'location', 'Latitude1':'lat', 'Longitude1':'lng', 'IncidentTypeDescription1':'indcident_desc'})

In [4]:
tornado_calls.time = pd.to_datetime(tornado_calls.time)
tornado_incidents.time = pd.to_datetime(tornado_incidents.time)

In [5]:
tornado_calls.dropna(inplace=True)
tornado_incidents.dropna(inplace=True)

In [6]:
tornado_calls.sort_values(by= 'time')

,time,call,lat,lng,tower_addy
0,2020-03-03 00:34:00,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector
1,2020-03-03 00:36:00,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE
2,2020-03-03 00:36:00,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector
4,2020-03-03 00:36:00,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N
5,2020-03-03 00:37:00,911 Calls,36.149032,-86.795747,WIRELESS CALLER
...,...,...,...,...,...
471,2020-03-03 04:54:00,911 Calls,36.189415,-86.813557,WIRELESS CALLER
470,2020-03-03 04:54:00,911 Calls,36.142552,-86.635330,WIRELESS CALLER
472,2020-03-03 04:55:00,911 Calls,36.223534,-86.757874,WIRELESS CALLER
473,2020-03-03 04:56:00,911 Calls,36.168398,-86.783774,600 DR MARTIN L KING JR BLVD - NE


In [27]:
heatmap = folium.Map(location = [36.1627, -86.7816], 
                    zoom_start = 10, tiles = 'Stamen Toner', gradient={0.1: 'blue', 0.5: 'lime', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.4, 
                max_opacity=0.8)
heatmap

In [14]:
heatmap = folium.Map(location = [36.1627, -86.7816], 
                    zoom_start = 10, tiles = 'cartodbdark_matter')

tornado_calls['count'] = 1

tornado_calls['minute'] = pd.DatetimeIndex(tornado_calls['time']).minute
tornado_calls['second'] = pd.DatetimeIndex(tornado_calls['time']).second
tornado_calls['hour'] = pd.DatetimeIndex(tornado_calls['time']).hour

df2 = pd.DataFrame(
    tornado_calls.groupby(['tower_addy', 'lat', 'lng'])\
    ['count'].sum().sort_values(ascending = False))

lst = df2.groupby(
    ['lat', 'lng'])\
    .sum().reset_index().values.tolist()

In [ ]:
df2

In [26]:
HeatMap(data = lst, radius = 20).add_to(heatmap);
heatmap

In [28]:
m2 = folium.Map(location = [36.1627, -86.7816], 
                    zoom_start = 10, tiles = 'Stamen Toner', gradient={0.1: 'blue', 0.5: 'lime', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.4, 
                max_opacity=0.8)

In [29]:
lat_long_list = []
for i in tornado_calls['time'].unique():
    temp=[]
    for index, instance in tornado_calls[tornado_calls['time'] == i].iterrows():
        temp.append([instance['lat'],instance['lng']])
    lat_long_list.append(temp)

In [30]:
#converting it to datetime format
tornado_calls['time']= pd.to_datetime(tornado_calls['time'])

#creating a time index
time_index1 = []
for i in tornado_calls['time'].unique():
    time_index1.append(i)

#formatting the index
date_strings = [pd.to_datetime(d).strftime('%d/%m/%Y, %H:%M:%S') for d in time_index1]




In [31]:
HeatMapWithTime(lat_long_list,radius=30, auto_play=True, position='bottomright', name="cluster", index=date_strings, max_opacity=0.7).add_to(m2)
m2

## timelapse_heat = []

for hour in tornado_calls['hour'].sort_values().unique(): 
    timelapse_heat.append(tornado_calls.loc[tornado_calls['hour'] == hour, # for each hour append to list  
    ['lat', 'lng', 'count']]\
    .groupby(['lat', 'lng'])\
    .sum().reset_index().values.tolist())
    
timelapse_heat



heatmap2 = folium.Map(location = [36.1627, -86.7816], 
                    zoom_start = 10)


start = pd.datetime(2020,3,3,0)

end = pd.datetime(2020,3,3,23)

daterange = pd.date_range(start=start,
                          end=end,
                         periods=24)

time_index = [d.strftime("%I:%M %p") for d in daterange]

HeatMapWithTime(timelapse_heat,radius=11,
                index = time_index,
                gradient={0.1: 'blue', 0.5: 'lime', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.4, 
                max_opacity=0.8, 
                use_local_extrema=True)\
                .add_to(heatmap2)

In [ ]:
date_strings

In [ ]:
tornado_calls.info()

In [ ]:
calls_grouped = tornado_calls.groupby([pd.Grouper(key = 'time', freq='1h')]).count().reset_index()

In [ ]:
calls_grouped

In [ ]:
m2.save('../maps/m2.html')

In [32]:
calls_grouped1 = tornado_calls.groupby([pd.Grouper(key = 'time', freq='15min')]).count().reset_index()
calls_grouped1

,time,call,lat,lng,tower_addy,count,minute,second,hour
0,2020-03-03 00:30:00,50,50,50,50,50,50,50,50
1,2020-03-03 00:45:00,43,43,43,43,43,43,43,43
2,2020-03-03 01:00:00,44,44,44,44,44,44,44,44
3,2020-03-03 01:15:00,51,51,51,51,51,51,51,51
4,2020-03-03 01:30:00,49,49,49,49,49,49,49,49
5,2020-03-03 01:45:00,43,43,43,43,43,43,43,43
6,2020-03-03 02:00:00,37,37,37,37,37,37,37,37
7,2020-03-03 02:15:00,20,20,20,20,20,20,20,20
8,2020-03-03 02:30:00,15,15,15,15,15,15,15,15
9,2020-03-03 02:45:00,8,8,8,8,8,8,8,8
